In [7]:
import numpy as np
from PIL import Image
from PIL import ImageEnhance
import os

data_root = '../data/CephalometricLandmark'
raw_root = data_root + '/RawImage'
txt_root = data_root + '/AnnotationsByMD'
cropped_root = data_root + '/CroppedImage'
contrast_root = data_root + "/ContrastImage"
contrast1_root = data_root + "/ContrastImage1"

In [8]:
# mkdir 
dir_path = contrast1_root
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
    
for i in range(19):
    if not os.path.exists(dir_path + "/%02d" % (i + 1)):
        os.mkdir(dir_path + "/%02d" % (i + 1))


In [ ]:

x = [[] for i in range(19)]
y = [[] for i in range(19)]
width = 640
height = 640

for i in range(1, 151):
    txt_filename1 = txt_root + '/400_senior' + "/%03d.txt" % i
    # txt_filename1 = './400_senior' + "/%03d.txt" % i

    with open(txt_filename1, 'r') as f:
        txts = f.read().split()
    for j in range(19):
        x[j].append(int(txts[j].split(',')[0]))
        y[j].append(int(txts[j].split(',')[1]))
for j in range(19):
    minx = np.min(x[j])
    miny = np.min(y[j])
    maxx = np.max(x[j])
    maxy = np.max(y[j])

    print(maxx - minx, maxy - miny)

    x0 = int(minx - (width - (maxx - minx)) / 2)
    x1 = int(maxx + (width - (maxx - minx)) / 2)

    y0 = int(miny - (height - (maxy - miny)) / 2)
    y1 = int(maxy + (height - (maxy - miny)) / 2)

    print(x0, y0)
    print(x1, y1)

    for index in range(1, 151):
        image_path = raw_root + '/TrainingData'
        image_filename = image_path + "/%03d.bmp" % index
        im = Image.open(image_filename)
        region = im.crop((x0, y0, x1, y1))
        region.save(cropped_root + "/%02d/%03d.bmp" % (j + 1, index))

    for index in range(151, 301):
        image_path = raw_root + '/Test1Data'
        image_filename = image_path + "/%03d.bmp" % index
        im = Image.open(image_filename)
        region = im.crop((x0, y0, x1, y1))
        region.save(cropped_root + "/%02d/%03d.bmp" % (j + 1, index))

    for index in range(301, 401):
        image_path = raw_root + '/Test2Data'
        image_filename = image_path + "/%03d.bmp" % index
        im = Image.open(image_filename)
        region = im.crop((x0, y0, x1, y1))
        region.save(cropped_root + "/%02d/%03d.bmp" % (j + 1, index))


In [1]:
from mxnet import image
image_filename = raw_root + '/TrainingData/001.bmp'
print(image.imread(image_filename).shape)


(2400, 1935, 3)


In [9]:
# 图片增强

for i in range(19):
    for j in range(0,400):
        src_file = cropped_root +  "/%02d/%03d.bmp" % (i + 1, j + 1)
        dst_file = contrast1_root + "/%02d/%03d.bmp" % (i + 1, j + 1)
        image = Image.open(src_file)
        enh_con = ImageEnhance.Contrast(image)
        contrast = 0.5
        image_contrasted = enh_con.enhance(contrast)
        image_contrasted.save(dst_file)